### Mount Drive

In [ ]:
!sudo add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!sudo apt-get update -qq 2>&1 > /dev/null
!sudo apt -y install -qq google-drive-ocamlfuse 2>&1 > /dev/null
!google-drive-ocamlfuse



debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
/usr/bin/xdg-open: 851: /usr/bin/xdg-open: www-browser: not found
/usr/bin/xdg-open: 851: /usr/bin/xdg-open: links2: not found
/usr/bin/xdg-open: 851: /usr/bin/xdg-open: elinks: not found
/usr/bin/xdg-open: 851: /usr/bin/xdg-open: links: not found
/usr/bin/xdg-open: 851: /usr/bin/xdg-open: lynx: not found
/usr/bin/xdg-open: 851: /usr/bin/xdg-open: w3m: not found
xdg-open: no method available for opening 'https://accounts.google.com/o/oauth2/auth?client_id=564921029129.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fgd-ocaml-auth.appspot.

In [ ]:
!sudo apt-get install -qq w3m # to act as web browser 
!xdg-settings set default-web-browser w3m.desktop # to set default browser
%cd /content
!mkdir drive
%cd drive
!mkdir MyDrive
%cd ..
%cd ..
!google-drive-ocamlfuse /content/drive/MyDrive

debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package w3m.
(Reading database ... 123947 files and directories currently installed.)
Preparing to unpack .../w3m_0.5.3-36build1_amd64.deb ...
Unpacking w3m (0.5.3-36build1) ...
Setting up w3m (0.5.3-36build1) ...
Processing triggers for mime-support (3.60ubuntu1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
/content
/content/drive
/content
/
Access token retrieved correctly.


### Import Lib

In [ ]:
%%capture
!pip install transformers

In [ ]:
%%capture
!pip install stanza

In [ ]:
from zipfile import ZipFile
from collections import defaultdict
from collections import Counter
import pandas as pd
import pickle
from collections import defaultdict
from tqdm import tqdm
import stanza
stanza.download('en')
st = stanza.Pipeline(lang='en', processors='tokenize')
import copy
import torch
import numpy as np
import numpy as np
import random
import torch
import math
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm

INFO:stanza:Downloading default packages for language: en (English) ...


INFO:stanza:Finished downloading models and saved to /root/stanza_resources.
INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Loading these models for language: en (English):
| Processor | Package  |
------------------------
| tokenize  | combined |

INFO:stanza:Use device: gpu
INFO:stanza:Loading: tokenize
INFO:stanza:Done loading processors!


###Load UMLS

#### Load concepts names

In [ ]:
with open('/content/drive/MyDrive/Dataset/UMLs/MRCONSO.RRF') as f:
  data = f.read().split('\n')
data = data[:-1]

umls = defaultdict(list)

for row in data:
  row = row.split('|')
  umls[row[0]].append(row[-5])

print(f'There are {len(umls)} concepts in UMLS')

There are 3465486 concepts in UMLS


In [ ]:
entities = []
for key, values in umls.items():
  umls[key] = list(set(values))
  entities += umls[key]

In [ ]:
counter = dict(Counter(entities)) # {e_1: f_1; e_2: f_2; ....}
value_count = counter.values()
value_count = list(value_count) 

counter_value = dict(Counter(value_count)) # {f_1: ff_1; f_2: ff_2;...}
sum_count = len(counter) #sum(counter_value.values())
statistic = {}

for key, value in counter_value.items():
  statistic[key] = round(value/sum_count,3)

df = pd.DataFrame([statistic])
df = pd.melt(df, value_vars=df.columns[0:])
df.head()

,variable,value
0,1,0.991
1,2,0.008
2,3,0.001
3,4,0.000
4,5,0.000


In [ ]:
train_data = []
for row in data:
  tmp = row.split('\t')
  if len(tmp) == 6:
    if tmp[0] in training_pmids:
      train_data.append([tmp[3],tmp[-1].split(':')[1]])

unknown_entities = []
for item in train_data:
  if item[1] not in umls:
    unknown_entities.append(item[1])

unknown_entities = set(unknown_entities)
print(f'There are {len(unknown_entities)} unknown enities in train set')


dev_data = []
for row in data:
  tmp = row.split('\t')
  if len(tmp) == 6:
    if tmp[0] in dev_pmids:
      dev_data.append([tmp[3],tmp[-1].split(':')[1]])

unknown_entities = []
for item in dev_data:
  if item[1] not in umls:
    unknown_entities.append(item[1])

unknown_entities = set(unknown_entities)
print(f'There are {len(unknown_entities)} unknown enities in dev set')

test_data = []
for row in data:
  tmp = row.split('\t')
  if len(tmp) == 6:
    if tmp[0] in test_pmids:
      test_data.append([tmp[3],tmp[-1].split(':')[1]])

unknown_entities = []
for item in test_data:
  if item[1] not in umls:
    unknown_entities.append(item[1])

unknown_entities = set(unknown_entities)
print(f'There are {len(unknown_entities)} unknown enities in test set')

There are 0 unknown enities in train set
There are 0 unknown enities in dev set
There are 0 unknown enities in test set


#### Load semantic type

In [ ]:
with open('/content/drive/MyDrive/Dataset/UMLs/MRSTY.RRF') as f:
  data = f.read().split('\n')
data = data[:-1]

semantic_types = defaultdict(str)
for row in data:
  row = row.split('|')
  semantic_types[row[0]] = row[-4]

### Process ST21pv

#### Load St21pv

In [ ]:
with open('/content/drive/MyDrive/Dataset/ST21pv/corpus_pubtator.txt') as f:
  data = f.read().split('\n\n')
data = data[:-1]

corpus_st21pv = []

for item in data:

  arr = item.split('\n')
  id = arr[0].split('|')[0]
  title = arr[0][11:]
  abstract = arr[1][11:]

  tmp = id + '\n' + title + '\n' + abstract + '\n'

  for row in arr[2:]:
    splits = row.split('\t')
    tmp += '\t'.join(splits[1:4] + [splits[5]]) + '\n'
  tmp = tmp[:-1]
  corpus_st21pv.append(tmp)

### Delete overlapping mentions

In [ ]:
count = 0

deleted_terms = []

for item in corpus_st21pv:
  arr = item.split('\n')
  
  id = arr[0]
  text = arr[1] + '\n' + arr[2]
  entities = arr[3:]

  positions = []
  for entity in entities:
    splits = entity.split('\t')
    s = splits[0]
    e = splits[1]
    positions.append([s,e, splits[2], splits[3],id])
  
  i = 1
  while i < len(positions):
    if int(positions[i][0]) < int(positions[i-1][1]):
      if int(positions[i][0]) == int(positions[i-1][0]) and int(positions[i-1][1]) < int(positions[i][1]):
        count += 1
        deleted_terms.append(positions[i-1])
        positions.pop(i-1)
        continue
      else:
        #if int(positions[i][0]) == int(positions[i-1][0]) and int(positions[i-1][1]) == int(positions[i][1]):
        #  i+=1
        #  continue
        #else:
        count += 1
        deleted_terms.append(positions[i])
        positions.pop(i)
    else:
      i += 1

    

new_corpus_st21pv = []
count = 0
term_deleted = []
for item in corpus_st21pv:
  arr = item.split('\n')
  id = arr[0]
  
  text = arr[1] + '\n' + arr[2]
  entities = arr[3:]
  new_entities = []

  for entity in entities:
    splits = entity.split('\t')
    s = splits[0]
    e = splits[1]
    tmp = [s,e, splits[2], splits[3],id]
    if tmp in deleted_terms:
      count += 1
      term_deleted.append(entity.split('\t') + [id])
      deleted_terms.remove(tmp)
    else: 
      new_entities.append(entity)

  tmp = id+ '\n' + text + '\n'
  for i in new_entities:
    tmp += i + '\n'
  tmp = tmp[:-1]
  new_corpus_st21pv.append(tmp)

print(f'Have deleted {count} mentions')
corpus_st21pv = new_corpus_st21pv

404
Have deleted 404 mentions


### Detect abbreviations in ST21pv

In [ ]:
with open("/content/drive/MyDrive/Dataset/ST21pv/out.txt") as f:
  data = f.read().split('\n\n')

In [ ]:
abbreviation_dict = defaultdict(list)

for item in data:
  arr = item.split('\n')
  id = arr[0]
  for row in arr[1:]:
    if '|' in row and len(row.split('|')) == 3:
      row = row.split('|')
      abbreviation_dict[id].append([row[0].replace('\t',''), row[1]])

### Replace abbreviations with long form

In [ ]:
corpus_st21pv_lf = []

for item in corpus_st21pv:
  arr = item.split('\n')
  id = arr[0]
  
  if id not in abbreviation_dict:
    corpus_st21pv_lf.append(item)   
    continue
  
  title = arr[1] + "."
  len_title = len(title)

  abstract = arr[2]
  entities = arr[3:]

  text = title + '\n' + abstract
  
  list_entities = []

  add = 0
  flag = True
  
  for i,entity in enumerate(entities):
    
    entity = entity.split('\t')
    if i >= 1 and entities[i].split('\t')[:-1] == entities[i-1].split('\t')[:-1]:
      tmp = '\t'.join(list_entities[-1].split('\t')[:-1]) + '\t' + entity[-1] + '\n'
      list_entities.append(tmp)
      continue
    name = entity[2]
    new_name = name
    s = entity[0]
    e = entity[1]
    if int(s) >= len_title:
      s = str(int(s) + 1)
      e = str(int(s) + 1)

    values = abbreviation_dict[id]
    for pair in values:
      if name == pair[0]:
        new_name = pair[1]
        break

    if flag == False:
      s = str(int(s) + add)
    else:
      flag = False
  

    text = text[:int(s)] + new_name + text[int(s)+len(name):] 
    add += len(new_name) - len(name)
    e = str(int(s) + len(new_name))
    list_entities.append(s+'\t'+e+'\t' + new_name +'\t' + entity[-1] + '\n')
    #print(text)
  tmp = id + '\n' + text + '\n'
  for i in list_entities:
    tmp += i
  tmp = tmp[:-1]
  corpus_st21pv_lf.append(tmp)
  tmp = ''
  list_entities = []


#### Test

In [ ]:
t = corpus_st21pv_lf[0]
print(t)

text = t.split('\n')[1] + '\n' + t.split('\n')[2]
entities = t.split('\n')[3:]

print('--------------------------------')
for entity in entities:
  entity = entity.split('\t')
  print(entity[0],'\t',entity[1], '\t', text[int(entity[0]): int(entity[1])], '\t', entity[2])

25763772
dynactin 4 as a modifier of chronic Pseudomonas aeruginosa infection in cystic fibrosis.
Pseudomonas aeruginosa (Pa) infection in cystic fibrosis (cystic fibrosis) patients is associated with worse long-term pulmonary disease and shorter survival, and chronic Pa infection (chronic Pa infection) is associated with reduced lung function, faster rate of lung decline, increased rates of exacerbations and shorter survival. By using exome sequencing and extreme phenotype design, it was recently shown that isoforms of dynactin 4 (dynactin 4) may influence Pa infection in cystic fibrosis, leading to worse respiratory disease. The purpose of this study was to investigate the role of dynactin 4 missense variants on Pa infection incidence, age at first Pa infection and chronic Pa infection incidence in a cohort of adult cystic fibrosis patients from a single centre. Polymerase chain reaction and direct sequencing were used to screen DNA samples for dynactin 4 variants. A total of 121 adu

#### Check

In [ ]:
def check_abbreviation_replace_correct(corpus):
  flag = True
  for i,item in enumerate(corpus):
    arr = item.split('\n')
    text = arr[1] + '\n' + arr[2]
    entities = arr[3:]
    id = arr[0]

    for entity in entities:
      tmp = entity.split('\t')
      s = int(tmp[0])
      e = int(tmp[1])
      name = tmp[2]

      if text[s:e] != name:
        flag = False
        print(s,'\t',e,i)
  
  if flag == True:
    print('There is no error detected!')

In [ ]:
check_abbreviation_replace_correct(corpus_st21pv_lf)

There is no error detected!


In [ ]:
with open("/content/drive/MyDrive/models/clustering_based_inference_bio_entity_linking/all_mentions.txt", 'w') as f:
  for item in corpus_st21pv_lf:
    rows = item.split('\n')
    entities = rows[3:]

    for entity in entities:
      entity = entity.split('\t')
      f.write(rows[0] + '||' + entity[2] + '||' + entity[-1].split(':')[-1] + '\n')

### Convert to IOB2 format

In [ ]:
IOB2_tags = defaultdict(list)
count = 0
for item in tqdm(corpus_st21pv_lf):
  arr = item.split('\n') 
  id = arr[0]
  text = arr[1] + '\n' + arr[2]
  entities = arr[3:]

  list_entities = []
  for entity in entities:
    splits = entity.split('\t')
    s = int(splits[0])
    e = int(splits[1])
    name = splits[2]
    cui = splits[3].split(':')[1]

    list_entities.append([s,e,name, cui])

  doc = st(text)

  list_sents = []


  for sentence in doc.sentences:
    list_tokens = []
    exist_mention = False
    for token in sentence.tokens:
      string = token.text
      start_c = token.start_char
      end_c = token.end_char

      flag = False
      for entity in list_entities:
        if entity[0] == start_c:
          count += 1
          flag = True
          list_tokens.append([string, 'B:'+ entity[-1]])
          exist_mention = True
          break
        elif entity[0] < start_c and entity[1] > start_c:
          exist_mention = True
          flag = True
          list_tokens.append([string, 'I:'+ entity[-1]])
          break
      if flag==False:
        list_tokens.append([string, 'O'])
    
    if exist_mention == True:
      list_sents.append(list_tokens)
  IOB2_tags[id] = list_sents

100%|██████████| 4392/4392 [02:12<00:00, 33.18it/s]


In [ ]:
with open("/content/drive/MyDrive/Dataset/ST21pv/IOB2_tags.txt", 'w') as f:
  for key, values in IOB2_tags.items(): # sentences
    f.write(key + '\n')
    
    for sentence in values:
      for token in sentence:
        f.write(token[0] + '\t' + token[1] + '\n')
      f.write('\n')  
    f.write('\n')